In [1]:
# pip install xgboost==1.7.1
# import xgboost as xgb
# print("Versión de XGBoost:", xgb.__version__)
# PREPROCESS
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve 
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
df=pd.read_csv(r'C:\Users\ARMANDO\Documents\data\dataset_full.csv')
#StandardScaler Standardize features by removing the mean and scaling to unit variance.The standard score of a sample x is calculated as:
#z = (x - u) / s
#varianza=1 mu=0
scaler = StandardScaler() # mi escalador de features numericos
boolean_features = ['email_in_url', 'domain_in_ip', 'server_client_domain', 'tld_present_params', 'domain_spf', 'tls_ssl_certificate', 'url_google_index', 'domain_google_index', 'phishing']
#el d1 es el dataset sin los booleans
d1=df.drop(boolean_features, axis=1)
#Como identificamos facilmente los booleans que son pocos,  solo falta quedarnos con el "antijoin" de los booleans
#sacamos columnas que no son booleanas y que son numéricas | #"Return a new Index with elements of index not in other.This is the set difference of two Index objects." osea  no en booleans
numcols=d1.columns.difference(boolean_features)
#escalamos todos los que no son booleanos "numcols ya es una lista" aplicamos fittransform
#fit_transform() generalmente se usa en transformadores como StandardScaler de scikit-learn 
#para ajustar la escala de las características y luego transformar los datos en consecuencia.
#fit: Se ajusta el transformador a los datos de entrenamiento. En el contexto de StandardScaler, 
# esto calcula la media y la desviación estándar de cada característica en los datos de entrenamiento.
#transfromr aplica la transformación a los datos de entrenamiento
# como son un metodo de mi scaler la fit-transformación en cuestión es un método de mi scaler (standarizador)

d1[numcols]=scaler.fit_transform(d1[numcols])
#al concatenar tenemos nuestros datos homogeneos
TrainDs=pd.concat([d1, df[boolean_features]], axis=1)
target=TrainDs['phishing']
#features=features.drop('phishing',axis=1 )
kBest = SelectKBest(score_func=f_classif, k=75)
Best_30 = kBest.fit_transform(TrainDs, target)
Best_30 = TrainDs.columns[kBest.get_support()].tolist()
scores=kBest.scores_
kBest = pd.DataFrame({'Feature': Best_30, 'Score': scores[kBest.get_support()]})
Best30 = kBest.sort_values(by= 'Score', ascending = False)[:30].reset_index(drop=True)
#Best30=Best30[1:].index
#corr_cols = Best30.index
corr_cols = Best30['Feature'].tolist()
x_train = TrainDs[corr_cols].drop('phishing', axis = 1)
y_train = TrainDs['phishing']
print('fin de preporcessor,  comienza xgboosting')
#PROCESSOR
#TRAIN
#test_size=0.2 significa que el 20% de los datos serán para el conjunto prueba mientras que el 80% para training
# Se utiliza comúnmente el número 42, pero podrías usar 
# cualquier número entero que desees. La idea es que al usar el mismo valor de random_state, 
# obtendrás siempre la misma división de datos
params = {'objective': 'binary:logistic','eval_metric': 'logloss'}
a, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

num_round = 100  # Número de iteraciones de entrenamiento

model = xgb.train(params, dtrain, num_round)
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(dtest)
# de probabilidades a binarios (0 o 1)
y_pred_binary = [1 if prob >= 0.5 else 0 for prob in y_pred]
# Calculando la precisión del modelo
accuracy = accuracy_score(y_test, y_pred_binary)
print('exito!')
print("Accuracy:", accuracy)
y_pred_binary

1

A continuación hace un dump del modelo y se empaqueta en un modelo tar_gz

In [143]:
import tarfile
import joblib

ruta_modelo = 'C:/Users/ARMANDO/Documents/Python Scripts/model.joblib'
# Guardar el modelo en un archivo Joblib
joblib.dump(model, ruta_modelo)

def compress_to_tar_gz(file_to_compress, output_tar_gz):
    with tarfile.open(output_tar_gz, "w:gz") as tar:
        tar.add(file_to_compress, arcname=file_to_compress.split("/")[-1])

# Ruta del archivo que quieres comprimir
file_to_compress = 'C:/Users/ARMANDO/Documents/Python Scripts/model.joblib'

# Ruta y nombre del archivo comprimido .tar.gz
output_tar_gz = 'C:/Users/ARMANDO/Documents/Python Scripts/model_29f.tar.gz'

# Comprimir el archivo
compress_to_tar_gz(file_to_compress, output_tar_gz)
